In [1]:
# pip install nltk tensorflow
# nltk.download('punkt')
# nltk.download('wordnet')

In [1]:
import nltk
import json
import pickle
import numpy as np
import random

from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '/', '@', '$']

In [3]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        print(words) 
        
        documents.append((w,intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(words)
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]   
# print(words)
words = sorted(list(set(words)))

classes = sorted(list(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# print(len(documents), "Documents: ", documents)
# print(len(classes), "Classes: ", classes)
# print(len(words), "Lemmatized Words: ", words)

['Hi', 'there']
['Hi', 'there', 'How', 'are', 'you']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Namaste']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Namaste', 'yo']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Namaste', 'yo', 'Bye']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Namaste', 'yo', 'Bye', 'See', 'you', 'later']
['Hi

In [7]:
#Training Data

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag,output_row])

random.shuffle(training)
training = np.array(training)



C:\Users\excal\AppData\Local\Temp\ipykernel_21024\3050057117.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
train_x = np.array(list(training[:, 0]))  
train_y = np.array(list(training[:, 1]))  
print(train_x)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
model = Sequential()
model.add(Dense(128, input_shape =(len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),  activation = 'softmax'))


In [10]:
sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics= ['accuracy'])

In [11]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs= 200, batch_size = 5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
25/25 [==============================] - 0s 1ms/step - loss: 3.1647 - accuracy: 0.0323
Epoch 2/200
25/25 [==============================] - 0s 875us/step - loss: 3.0981 - accuracy: 0.0968
Epoch 3/200
25/25 [==============================] - 0s 958us/step - loss: 3.0545 - accuracy: 0.0887
Epoch 4/200
25/25 [==============================] - 0s 875us/step - loss: 3.0016 - accuracy: 0.0968
Epoch 5/200
25/25 [==============================] - 0s 917us/step - loss: 2.8858 - accuracy: 0.1371
Epoch 6/200
25/25 [==============================] - 0s 875us/step - loss: 2.8078 - accuracy: 0.1452
Epoch 7/200
25/25 [==============================] - 0s 917us/step - loss: 2.7613 - accuracy: 0.1935
Epoch 8/200
25/25 [==============================] - 0s 875us/step - loss: 2.5702 - accuracy: 0.3306
Epoch 9/200
25/25 [==============================] - 0s 917us/step - loss: 2.4195 - accuracy: 0.3145
Epoch 10/200
25/25 [==============================] - 0s 875us/step - loss: 2.3700 - accuracy

25/25 [==============================] - 0s 958us/step - loss: 0.1320 - accuracy: 0.9758
Epoch 82/200
25/25 [==============================] - 0s 1ms/step - loss: 0.1318 - accuracy: 0.9516
Epoch 83/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0936 - accuracy: 0.9839
Epoch 84/200
25/25 [==============================] - 0s 1000us/step - loss: 0.0976 - accuracy: 0.9677
Epoch 85/200
25/25 [==============================] - 0s 958us/step - loss: 0.1017 - accuracy: 0.9839
Epoch 86/200
25/25 [==============================] - 0s 958us/step - loss: 0.1577 - accuracy: 0.9597
Epoch 87/200
25/25 [==============================] - 0s 917us/step - loss: 0.1738 - accuracy: 0.9597
Epoch 88/200
25/25 [==============================] - 0s 958us/step - loss: 0.1664 - accuracy: 0.9516
Epoch 89/200
25/25 [==============================] - 0s 917us/step - loss: 0.1423 - accuracy: 0.9597
Epoch 90/200
25/25 [==============================] - 0s 875us/step - loss: 0.0931 - accuracy: 0.9

25/25 [==============================] - 0s 875us/step - loss: 0.1074 - accuracy: 0.9758
Epoch 161/200
25/25 [==============================] - 0s 917us/step - loss: 0.0540 - accuracy: 0.9839
Epoch 162/200
25/25 [==============================] - 0s 833us/step - loss: 0.0593 - accuracy: 0.9839
Epoch 163/200
25/25 [==============================] - 0s 958us/step - loss: 0.1142 - accuracy: 0.9677
Epoch 164/200
25/25 [==============================] - 0s 917us/step - loss: 0.0516 - accuracy: 0.9758
Epoch 165/200
25/25 [==============================] - 0s 917us/step - loss: 0.0869 - accuracy: 0.9839
Epoch 166/200
25/25 [==============================] - 0s 875us/step - loss: 0.0341 - accuracy: 0.9919
Epoch 167/200
25/25 [==============================] - 0s 875us/step - loss: 0.0584 - accuracy: 0.9839
Epoch 168/200
25/25 [==============================] - 0s 917us/step - loss: 0.0495 - accuracy: 0.9839
Epoch 169/200
25/25 [==============================] - 0s 958us/step - loss: 0.0322 - a